<a href="https://colab.research.google.com/github/SoohyeonB/Ewha-git/blob/main/trainNsave_(3).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers==3.0.2
!pip install torch==1.2.0

     |████████████████████████████████| 46.9 MB 46 kB/s 
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1
     |████████████████████████████████| 344 kB 5.3 MB/s 
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp37-cp37m-linux_x86_64.whl size=595736 sha256=47722bdc2a559822123c32093c898a486366a706c2c5165d3ce70bcca415d757
  Stored in directory: /root/.cache/pip/wheels/be/b4/06/7f3fdfaf707e6b5e98b79c041e023acffbe395d78a527eae00
Successfully built gluonnlp
     |████████████████████████████████| 1.2 MB 5.4 MB/s 
     |████████████████████████████████| 769 kB 5.4 MB/s 
     |████████████████████████████████| 895 kB 32.8 MB/s 
     |████████████████████████████████| 3.0 MB 24.2 MB/s 
     |████████████████████████████████| 748.9 MB 638 bytes/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.9.0+cu111
    Uninstalling torch-1.9.0+cu111:
      Succes

In [3]:
# SKT KoBERT
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-4f60yqu4
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-4f60yqu4
  Created wheel for kobert: filename=kobert-0.1.2-py3-none-any.whl size=12771 sha256=dd50fccf619094e3dac279f25de90a62adf3e5d66d8c02726ac681db72eed5ab
  Stored in directory: /tmp/pip-ephem-wheel-cache-gzqe7ef7/wheels/d3/68/ca/334747dfb038313b49cf71f84832a33372f3470d9ddfd051c0
Successfully built kobert


In [4]:
%cd drive/MyDrive/코딩관련/colab/nlp/kobert_test/

[Errno 2] No such file or directory: 'drive/MyDrive/코딩관련/colab/nlp/kobert_test/'
/content


In [5]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook
import pandas as pd

#kobert
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

#transformers
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

#GPU 사용 시
device = torch.device("cuda:0")


#device = torch.device("cpu")

In [6]:
#BERT 모델, Vocabulary 불러오기
bertmodel, vocab = get_pytorch_kobert_model()

ConnectionError: ignored

In [ ]:
#구글드라이브 연동
# from google.colab import drive
# drive.mount('/content/drive')

chatbot_data = pd.read_excel('data_leafNbug.xlsx')

# answer json 파일 열기

import json
from google.colab import files
with open('leafNbug_answer.json') as json_file:
  answer_data = json.load(json_file)



In [ ]:
chatbot_data.sample(n=10)

In [ ]:
print(answer_data)

In [ ]:
# 문장에서 특수문자 제거하기
import re
def cleanText(input):
  clean = re.sub('[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》]','',input)

  return clean

In [ ]:
# 네이버 맞춤법 교정기 기반 라이브러리
!pip install git+https://github.com/ssut/py-hanspell.git

from hanspell import spell_checker

def spellCheck(input):
  checked = spell_checker.check(input)
  #print(checked.as_dict())
  #print(checked.as_dict()['checked'])
  return checked.as_dict()['checked']


In [ ]:
chatbot_data.loc[(chatbot_data['클래스'] == "분갈이_방법"), '클래스'] = 0  
chatbot_data.loc[(chatbot_data['클래스'] == "분갈이_시기"), '클래스'] = 1  
chatbot_data.loc[(chatbot_data['클래스'] == "분갈이_이후"), '클래스'] = 2  
chatbot_data.loc[(chatbot_data['클래스'] == "분갈이_화분"), '클래스'] = 3  
chatbot_data.loc[(chatbot_data['클래스'] == "분갈이_흙"), '클래스'] = 4  
chatbot_data.loc[(chatbot_data['클래스'] == "잎의질병_화상"), '클래스'] = 5 
chatbot_data.loc[(chatbot_data['클래스'] == "재배환경_온도"), '클래스'] = 6
chatbot_data.loc[(chatbot_data['클래스'] == "재배환경_인조조명"), '클래스'] = 7
chatbot_data.loc[(chatbot_data['클래스'] == "재배환경_장소"), '클래스'] = 8
chatbot_data.loc[(chatbot_data['클래스'] == "재배환경_햇빛"), '클래스'] = 9
chatbot_data.loc[(chatbot_data['클래스'] == "튜토리얼_방법"), '클래스'] = 10
chatbot_data.loc[(chatbot_data['클래스'] == "물주기"), '클래스'] = 11 
chatbot_data.loc[(chatbot_data['클래스'] == "물주기_저면관수"), '클래스'] = 12 
chatbot_data.loc[(chatbot_data['클래스'] == "물주기_계절"), '클래스'] = 13
chatbot_data.loc[(chatbot_data['클래스'] == "번식_잎꽂이방법"), '클래스'] = 14 
chatbot_data.loc[(chatbot_data['클래스'] == "번식_자구"), '클래스'] = 15
chatbot_data.loc[(chatbot_data['클래스'] == "번식_적심"), '클래스'] = 16
chatbot_data.loc[(chatbot_data['클래스'] == "뿌리줄기_무름병"), '클래스'] = 17
chatbot_data.loc[(chatbot_data['클래스'] == "뿌리줄기_썩음병"), '클래스'] = 18
chatbot_data.loc[(chatbot_data['클래스'] == "뿌리줄기_기타"), '클래스'] = 19
chatbot_data.loc[(chatbot_data['클래스'] == "웃자람_증상"), '클래스'] = 20
chatbot_data.loc[(chatbot_data['클래스'] == "웃자람_해결"), '클래스'] = 21 
chatbot_data.loc[(chatbot_data['클래스'] == "잎의질병_곰팡이"), '클래스'] = 22
chatbot_data.loc[(chatbot_data['클래스'] == "잎의질병_과습"), '클래스'] = 23
chatbot_data.loc[(chatbot_data['클래스'] == "잎의질병_과습여부"), '클래스'] = 24
chatbot_data.loc[(chatbot_data['클래스'] == "잎의질병_마름"), '클래스'] = 25
chatbot_data.loc[(chatbot_data['클래스'] == "잎의질병_원인알수없음"), '클래스'] = 26
chatbot_data.loc[(chatbot_data['클래스'] == "잎의질병_탄저병"), '클래스'] = 27
chatbot_data.loc[(chatbot_data['클래스'] == "해충_기타"), '클래스'] = 28
chatbot_data.loc[(chatbot_data['클래스'] == "해충_깍지"), '클래스'] = 29
chatbot_data.loc[(chatbot_data['클래스'] == "해충_응애"), '클래스'] = 30
chatbot_data.loc[(chatbot_data['클래스'] == "해충_해충약"), '클래스'] = 31
 

data_list = []
for ques, label in zip(chatbot_data['질문 내용'], chatbot_data['클래스'])  :
    data = []
    ques = cleanText(ques)
    try:
      ques = spellCheck(ques)
    except:
        print(ques)
    #ques = spellCheck(ques)
    data.append(ques)
    data.append(str(label))

    data_list.append(data)

In [ ]:
print(data_list[5])
print(data_list[20])
print(data_list[40])

In [ ]:
#train & test 데이터로 나누기
from sklearn.model_selection import train_test_split
                                                         
dataset_train, dataset_test = train_test_split(data_list, test_size=0.25, random_state=0)

In [ ]:
print(len(dataset_train))
print(len(dataset_test))

In [ ]:
print(dataset_test[0])
print(dataset_test[1])
print(dataset_test[2])

In [ ]:
import pandas as pd
#dataset_test_temp = pd.DataFrame(dataset_test)
#dataset_test_temp.to_excel(excel_writer='sample_test.xlsx')

#dataset_train_temp = pd.DataFrame(dataset_train)
#dataset_train_temp.to_excel(excel_writer='sample_train.xlsx')


In [ ]:
df = pd.read_excel('sample_test.xlsx')
#dataset_test = df.values.tolist()
print(dataset_test[0])
print(dataset_test[1])
print(dataset_test[2])

df = pd.read_excel('sample_train.xlsx')
#dataset_train = df.values.tolist()
print(dataset_train[0])
print(dataset_train[1])
print(dataset_train[2])

In [ ]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [ ]:
# Setting parameters
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 10  #에폭
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [ ]:
#토큰화
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

In [ ]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)


In [ ]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=10,   ##클래스 수 조정##
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [ ]:
#BERT 모델 불러오기
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

#optimizer와 schedule 설정
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

#정확도 측정을 위한 함수 정의
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc
    
train_dataloader

In [ ]:
train_history=[]
test_history=[]
loss_history=[]
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
            train_history.append(train_acc / (batch_id+1))
            loss_history.append(loss.data.cpu().numpy())
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    #train_history.append(train_acc / (batch_id+1))
    
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))
    test_history.append(test_acc / (batch_id+1))

In [ ]:
import random 

def getAnswer(index, num):
  #answers=[]
  for tg in answer_data["intents"]:
    if tg['tag']==str(index):
      if num == 0:
        answers = tg['answer']
      elif num==1:
        answers = tg['answeryes']
      elif num==2:
        answers = tg['answerno']


  #ans = random.choice(answers)
  ans=answers[0]
  return ans

In [ ]:
# 수정본
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)
 
def predict(predict_sentence,beforeidx):

    predict_sentence = spellCheck(predict_sentence)
    predict_sentence = cleanText(predict_sentence)

    if predict_sentence=='예':
        answer = getAnswer(beforeidx,1)
        return answer,-1
    elif predict_sentence == '아니오':
        answer = getAnswer(beforeidx,2)
        return answer,-1
    else:
        #print("전처리 후: "+predict_sentence)
    
        data = [predict_sentence, '0']
        #print(data) #['물 줄까', '0']

        dataset_another = [data]
        #print(dataset_another) #[['물 줄까', '0']]

        another_test = BERTDataset(dataset_another, 0, 1, tok, max_len, True, False)
    
        test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=2) 
    
        model.eval()

        for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
            token_ids = token_ids.long().to(device)
            segment_ids = segment_ids.long().to(device)

            valid_length= valid_length
            label = label.long().to(device)

        
        
            out = model(token_ids, valid_length, segment_ids)
            tmp = out.detach().cpu().numpy()
            maxProb = np.max(tmp[0])
            test_eval=[]
            idx = -1
            for i in out:
                logits=i
                logits = logits.detach().cpu().numpy()

                idx = np.argmax(logits)

                if idx == 0:
                    test_eval.append("물주기")
                elif idx == 1:
                    test_eval.append("물주기_저면관수")
                    beforeidx=idx
                elif idx == 2:
                    test_eval.append("물주기_계절")
                    beforeidx=idx
                elif idx == 3:
                    test_eval.append("번식_잎꽂이방법")
                elif idx == 4:
                    test_eval.append("번식_자구")
                elif idx == 5:
                    test_eval.append("번식_적심")
                elif idx == 6:
                    test_eval.append("뿌리줄기_무름병")
                elif idx == 7:
                    test_eval.append("뿌리줄기_썩음병")
                elif idx == 8:
                    test_eval.append("뿌리줄기_기타")
                elif idx == 9:
                    test_eval.append("웃자람_증상")
                elif idx == 10:
                    test_eval.append("웃자람_해결")
                elif idx == 11:
                    test_eval.append("분갈이_방법")
                elif idx == 12:
                    test_eval.append("분갈이_시기")
                elif idx == 13:
                    test_eval.append("분갈이_화분")
                elif idx == 14:
                    test_eval.append("분갈이_흙")
                elif idx == 15:
                    test_eval.append("잎의질병_화상")
                elif idx == 16:
                    test_eval.append("재배환경_온도")
                elif idx == 17:
                    test_eval.append("재배환경_인조조명")
                elif idx == 18:
                    test_eval.append("재배환경_장소")
                elif idx == 19:
                    test_eval.append("재배환경_햇빛")
                elif idx == 20:
                    test_eval.append("튜토리얼_방법")
                elif idx == 21:
                    test_eval.append("잎의질병_곰팡이")
                elif idx == 22:
                    test_eval.append("잎의질병_과습")
                elif idx == 23:
                    test_eval.append("잎의질병_과습여부")
                elif idx == 24:
                    test_eval.append("잎의질병_마름")
                elif idx == 25:
                    test_eval.append("잎의질병_원인알수없음")
                elif idx == 26:
                    test_eval.append("잎의질병_탄저병")
                elif idx == 27:
                    test_eval.append("해충_기타")
                elif idx == 28:
                    test_eval.append("해충_깍지")
                elif idx == 29:
                    test_eval.append("해충_응애")                    
                elif idx == 30:
                    test_eval.append("해충_해충약")
                elif idx == 31:
                    test_eval.append("분갈이_이후")

                
           
            print(">> 질문 카테고리: " + test_eval[0])
             
            answer = getAnswer(idx,0)
            print(">> 답변: "+answer)

        return answer,idx

 



In [ ]:
#질문 무한반복하기! 0 입력시 종료
end = 1
while end == 1 :
    sentence = input("입력 : ")
    if sentence == 'quit' :
        break
    if sentence =='예':
        print(beforeidx)
        getAnswer(beforeidx,1)
    elif sentence =='아니오':
        getAnswer(beforeidx,2)
    else:
        predict(sentence)
    print("\n")

In [ ]:
!pip install flask==0.12.2
!pip install flask-ngrok

In [ ]:
from flask import Flask, request, jsonify
from flask_ngrok import run_with_ngrok
from flask import make_response
 
app = Flask(__name__)
run_with_ngrok(app)  # Start ngrok when app is run
#global beforeidx
#http://xxxxxx.ngrok.io/?msg=안녕
@app.route("/", methods=['GET','POST'])
def chatBot():
       global beforeidx 
       global s
       parameter_dict = request.args.to_dict()
       if len(parameter_dict) == 0:
           s="리피에게 궁금한걸 물어봐주세요~"
        
       else:
           question=request.args.get('msg')
           if question == '예':              
               s,tmp=predict(question,beforeidx)                
           elif question =='아니오':               
               s,tmp=predict(question,beforeidx)                
           else:
               s,beforeidx =predict(question,-1)
       result = json.dumps({"answer": s}, ensure_ascii=False)      
       res = make_response(result)     
       return res, 200,{'content-type': 'application/json'}       
        
        
        
 

if __name__ == '__main__':
    app.run()  # If address is in use, may need to terminate other sessions:
               # Runtime > Manage Sessions > Terminate Other Sessions